# Clasificación de sonido con Keras

## Dataset

Este conjunto de datos contiene los discursos de estos líderes prominentes; Benjamín Netanyahu, Jens Stoltenberg, Julia Gillard, Margaret
Tacher y Nelson Mandela que también representa los nombres de las carpetas. Cada audio de la carpeta es un PCM de un segundo de duración con una velocidad de muestreo de 16000 hz codificado.
Una carpeta llamada background_noise contiene audios que no son discursos, pero que se pueden encontrar en el interior alrededor del entorno del orador, por ejemplo, la audiencia riendo o aplaudiendo. Se puede mezclar con el discurso mientras se entrena.

### Referencias

https://www.kaggle.com/kongaevans/speaker-recognition-dataset/

## Enunciado

El objetivo del ejercicio consiste en utilizar un perceptrón multicapa para la identificación de la persona que se encuentra hablando a partir de un audio de un segundo de duración.

In [1]:
# Definimos un conjunto de constantes con las rutas de las carpetas que contienen los audios de cada persona
import os

DATASET_ROOT = "16000_pcm_speeches"
BENJAMIN_DATA = os.path.join(DATASET_ROOT, "Benjamin_Netanyau")
JENS_DATA = os.path.join(DATASET_ROOT, "Jens_Stoltenberg")
JULIA_DATA = os.path.join(DATASET_ROOT, "Julia_Gillard")
MARGARET_DATA = os.path.join(DATASET_ROOT, "Magaret_Tarcher")
NELSON_DATA = os.path.join(DATASET_ROOT, "Nelson_Mandela")

## 1. Preparación del conjunto de datos

In [2]:
# Definimos una función para parsear nuestro conjunto de datos
import librosa
import numpy as np

def parse_dataset(dataset_paths):
    X = []
    y = []
    for index, dataset in enumerate(dataset_paths):
        print("[+] Parsing {} data...".format(dataset))
        for fname in os.listdir(dataset):
            wav, sr = librosa.load(os.path.join(dataset, fname), sr=None)
            D = librosa.amplitude_to_db(np.abs(librosa.stft(wav)), ref=np.max)
            X.append(D)
            y.append(index)
    return (X, y)

In [3]:
X, y = parse_dataset([BENJAMIN_DATA, JENS_DATA, JULIA_DATA, MARGARET_DATA, NELSON_DATA])

[+] Parsing 16000_pcm_speeches\Benjamin_Netanyau data...
[+] Parsing 16000_pcm_speeches\Jens_Stoltenberg data...
[+] Parsing 16000_pcm_speeches\Julia_Gillard data...
[+] Parsing 16000_pcm_speeches\Magaret_Tarcher data...
[+] Parsing 16000_pcm_speeches\Nelson_Mandela data...


## 2. División del conjunto de datos

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [8]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [9]:
print("Longitud subconjunto de entrenamiento: ", len(X_train))
print("Longitud subconjunto de validación: ", len(X_val))
print("Longitud subconjunto de pruebas: ", len(X_test))

Longitud subconjunto de entrenamiento:  6750
Longitud subconjunto de validación:  376
Longitud subconjunto de pruebas:  375


## 3. Construcción del modelo

In [10]:
from tensorflow.keras.utils import to_categorical

def prep_dataset(X, y, shape):
    X_prep = np.array(X).reshape((len(X), shape))
    X_prep = X_prep.astype('float32') / 255
    y_prep = to_categorical(np.array(y))
    return (X_prep, y_prep)

In [11]:
X_train_prep, y_train_prep = prep_dataset(X_train, y_train, 1025*32)
X_val_prep, y_val_prep = prep_dataset(X_val, y_val, 1025*32)
X_test_prep, y_test_prep = prep_dataset(X_test, y_test, 1025*32)

In [12]:
X_train_prep

array([[-0.14826408, -0.15834898, -0.19321086, ..., -0.3137255 ,
        -0.3137255 , -0.3137255 ],
       [-0.19783637, -0.23456387, -0.25210872, ..., -0.3137255 ,
        -0.3137255 , -0.28176862],
       [-0.23888263, -0.24554221, -0.24458963, ..., -0.3137255 ,
        -0.3137255 , -0.3137255 ],
       ...,
       [-0.14759357, -0.15274224, -0.17885748, ..., -0.3137255 ,
        -0.31233773, -0.27470422],
       [-0.21802326, -0.24991392, -0.23892723, ..., -0.3137255 ,
        -0.28422064, -0.24405412],
       [-0.15930112, -0.17322442, -0.16390556, ..., -0.3137255 ,
        -0.29085886, -0.2649805 ]], dtype=float32)

In [13]:
y_train_prep

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

### 3.1. Entrenamiento de una RNA sencilla

In [14]:
# Entrenamos un perceptrón multicapa
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(activation='logistic', hidden_layer_sizes=(10,), solver='sgd')
clf.fit(X_train_prep, y_train)

C:\Users\Hugo\anaconda3\envs\curso_deep_learning\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(10,), solver='sgd')

In [15]:
# Realizamos la predicción con el conjunto de datos de validación
y_pred = clf.predict(X_val_prep)

In [16]:
# Mostramos el accuracy resultante de la clasificación
from sklearn.metrics import accuracy_score

accuracy_score(y_val, y_pred)

0.9441489361702128

### 3.2. Entrenamiento de una RNA profunda

In [17]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [18]:
network = models.Sequential()

network.add(layers.Dense(300, activation='relu', input_shape=(1025*32,)))
network.add(layers.Dense(200, activation='relu'))
network.add(layers.Dense(100, activation='relu'))
network.add(layers.Dense(5, activation='softmax'))

In [19]:
network.compile(loss='categorical_crossentropy',
                optimizer='sgd',
                metrics=['accuracy', 'Precision'])

In [20]:
history = network.fit(X_train_prep, 
                      y_train_prep,
                      epochs=30,
                      validation_data=(X_val_prep, y_val_prep))

Epoch 1/30
211/211 [==============================] - 8s 35ms/step - loss: 1.3866 - accuracy: 0.4036 - precision: 0.6667 - val_loss: 1.0990 - val_accuracy: 0.5160 - val_precision: 0.6886
Epoch 2/30
211/211 [==============================] - 8s 36ms/step - loss: 1.0516 - accuracy: 0.5596 - precision: 0.7250 - val_loss: 0.7989 - val_accuracy: 0.6782 - val_precision: 0.8308
Epoch 3/30
211/211 [==============================] - 7s 33ms/step - loss: 0.8553 - accuracy: 0.6607 - precision: 0.7721 - val_loss: 0.6324 - val_accuracy: 0.8059 - val_precision: 0.9306
Epoch 4/30
211/211 [==============================] - 7s 33ms/step - loss: 0.7185 - accuracy: 0.7231 - precision: 0.7972 - val_loss: 0.5492 - val_accuracy: 0.8085 - val_precision: 0.8770
Epoch 5/30
211/211 [==============================] - 7s 32ms/step - loss: 0.6378 - accuracy: 0.7661 - precision: 0.8219 - val_loss: 0.4694 - val_accuracy: 0.8324 - val_precision: 0.8769
Epoch 6/30
211/211 [==============================] - 7s 31ms/ste

In [21]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(10, 7))
plt.grid(True)
plt.gca().set_ylim(0, 1.2)
plt.xlabel("epochs")
plt.show()

### 3.3. Predicción para nuevos ejemplos

In [22]:
# Realizamos la predicción para el subconjunto de pruebas
test_loss, test_acc, test_prec = network.evaluate(X_test_prep, y_test_prep)

12/12 [==============================] - 0s 11ms/step - loss: 0.0681 - accuracy: 0.9733 - precision: 0.9812


In [23]:
print('test_acc:', test_acc)
print('test_prec:', test_prec)

test_acc: 0.9733333587646484
test_prec: 0.9811828136444092
